In [4]:
import datetime
from orbit_predictor.sources import get_predictor_from_tle_lines
import ephem
from sgp4.api import Satrec

In [5]:
TLE_LINES = (
    "1 44713U 19074A   23001.33310695  .00000538  00000-0  55037-4 0  9996",
    "2 44713  53.0541   9.1431 0002180  39.3908 320.7239 15.06389052173400"
)

In [6]:
name = "STARLINK"
line1 = TLE_LINES[0]
line2 = TLE_LINES[1]

In [6]:
predictor = get_predictor_from_tle_lines(TLE_LINES)
L = predictor.get_position(datetime.datetime(2023,1,1,17,32,55))  

In [7]:
predictor.get_position(datetime.datetime.now())

Position(when_utc=datetime.datetime(2023, 5, 24, 9, 56, 49, 987064), position_ecef=(2785.3038860385927, -3217.8235568895525, -5470.226357090315), velocity_ecef=(4.8175716122521415, 5.771944398524003, -0.9437102787403271), error_estimate=None)

In [13]:
tle_rec = ephem.readtle(name, line1, line2)
tle_rec.compute()


### SGP4 lib

In [7]:
satellite=Satrec.twoline2rv(line1, line2)

In [8]:
satellite.epochyr

23

In [11]:
e, r, v = satellite.sgp4(satellite.jdsatepoch, satellite.jdsatepochF)

In [15]:
print("r = ", r)
print("v = ", v)

r =  (6837.10650415244, 1100.4015126004147, -0.00161258720425325)
v =  (-0.7321305505601278, 4.501135427880932, 6.067780802835581)


### Skyfield lib

In [23]:
from skyfield.api import load, wgs84, EarthSatellite, Topos
import numpy as np


In [19]:
ts = load.timescale()
satellite = EarthSatellite(line1, line2, 'STARLINK', ts)


In [21]:
geocentric = satellite.at(satellite.epoch)
print(geocentric.position.m)


[6842659.22522678 1065219.73043605  -15182.01079798]


In [24]:
mine = np.array([6835086.828726, 1107352.760631, 9550.439149])

In [27]:
sum((geocentric.position.m - mine)**2)**0.5

49439.129265730095